# Simple QAOA

by Felix Xu, 13/06/2023

This is for benchmarking the performance of various devices on simple QAOA.

The code is based on [TensorCircuit](https://tensorcircuit.readthedocs.io/en/latest/index.html), the TensorFLow kernel is used.

<u>**No GPUs should be used!!!**</u>

Just-in-time compiling this also not allowed.

Please directly "Run All" then collect the results in Section "Results".

## Code

In [1]:
import tensorcircuit as tc
import tensorflow as tf
from qiskit_finance.data_providers import RandomDataProvider
from functools import partial
from IPython.display import clear_output
import numpy as np
import time
import datetime

K = tc.set_backend("tensorflow")

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
def QUBO_from_portfolio(cov, mean, q, B, t):
    """convert portfolio parameters to a Q-matrix
    cov: n-by-n covariance numpy array
    mean: numpy array of means
    q: the risk preference of investor
    B: budget
    t: penalty factor
    """
    n = cov.shape[0]
    R = np.diag(mean)
    S = np.ones((n, n)) - 2 * B * np.diag(np.ones(n))

    Q = q * cov - R + t * S
    return Q

def QUBO_to_Ising(Q):
    # input is n-by-n symmetric numpy array corresponding to Q-matrix
    # output is the components of Ising Hamiltonian

    n = Q.shape[0]

    offset = np.triu(Q, 0).sum() / 2
    pauli_terms = []
    weights = -np.sum(Q, axis=1) / 2

    for i in range(n):
        term = np.zeros(n)
        term[i] = 1
        pauli_terms.append(term)

    for i in range(n - 1):
        for j in range(i + 1, n):
            term = np.zeros(n)
            term[i] = 1
            term[j] = 1
            pauli_terms.append(term)

            weight = Q[i][j] / 2
            weights = np.concatenate((weights, weight), axis=None)

    return pauli_terms, weights, offset

def QAOA_from_Ising(params, nlayers, pauli_terms, weights):
    nqubits = len(pauli_terms[0])
    c = tc.Circuit(nqubits)
    for i in range(nqubits):
        c.h(i)
    for j in range(nlayers):
        for k in range(len(pauli_terms)):
            term = pauli_terms[k]
            index_of_ones = []
            for l in range(len(term)):
                if term[l] == 1:
                    index_of_ones.append(l)
            if len(index_of_ones) == 1:
                c.rz(index_of_ones[0], theta=2 * weights[k] * params[2 * j])
            elif len(index_of_ones) == 2:
                c.exp1(
                    index_of_ones[0],
                    index_of_ones[1],
                    unitary=tc.gates._zz_matrix,
                    theta=weights[k] * params[2 * j],
                )
            else:
                raise ValueError("Invalid number of Z terms")

        for i in range(nqubits):
            c.rx(i, theta=params[2 * j + 1])  # mixing terms
    return c

def Ising_loss(c, pauli_terms, weights):
    loss = 0.0
    for k in range(len(pauli_terms)):
        term = pauli_terms[k]
        index_of_ones = []

        for l in range(len(term)):
            if term[l] == 1:
                index_of_ones.append(l)

        if len(index_of_ones) == 1:
            delta_loss = weights[k] * c.expectation_ps(z=[index_of_ones[0]])

        else:
            delta_loss = weights[k] * c.expectation_ps(
                z=[index_of_ones[0], index_of_ones[1]]
            )

        loss += delta_loss

    return K.real(loss)

def QAOA_loss(nlayers, pauli_terms, weights, params):
    c = QAOA_from_Ising(params, nlayers, pauli_terms, weights)
    return Ising_loss(c, pauli_terms, weights)

def QAOA_solve(pauli_terms, weights, nlayers, iterations):
    print_every = 100
    learning_rate = 1e-2

    loss_val_grad = K.value_and_grad(partial(QAOA_loss, nlayers, pauli_terms, weights))
    loss_val_grad_jit = K.jit(loss_val_grad)

    opt = K.optimizer(tf.keras.optimizers.Adam(learning_rate))

    params = K.implicit_randn(shape=[2 * nlayers], stddev=0.5)
    for i in range(iterations):
        loss, grads = loss_val_grad_jit(params)
        params = opt.update(grads, params)

    return params

In [3]:
n_loops = 10
seed = 123
q = 0.5
penalty = 3
result = {}
iterations = 1000

for num_assets in [8, 9]:  # change here for num of qubits
    nlayers = num_assets * 2
    budget = np.floor(num_assets * 0.75)

    stocks = [("TICKER%s" % i) for i in range(num_assets)]
    data = RandomDataProvider(
        tickers=stocks,
        start=datetime.datetime(2016, 1, 1),
        end=datetime.datetime(2016, 1, 30),
        seed=seed,
    )
    data.run()

    mu = data.get_period_return_mean_vector()
    sigma = data.get_period_return_covariance_matrix()
    Q = QUBO_from_portfolio(sigma, mu, q, budget, penalty)
    portfolio_pauli_terms, portfolio_weights, portfolio_offset = QUBO_to_Ising(Q)

    states = []
    for i in range(2**num_assets):
        states.append(f"{bin(i)[2:]:0>{num_assets}}")
    cost = []
    for selection in states:
        x = np.array([int(bit) for bit in selection])
        cost.append(np.dot(x, np.dot(Q, x)))
    loc = np.where(cost == np.min(cost))

    accuracy = 0

    total_time = []
    n_error = 0
    for loops in range(n_loops):

        try:
            time_start = time.time()
            final_params = QAOA_solve(
                portfolio_pauli_terms, portfolio_weights, nlayers, iterations
            )
            time_end = time.time()
            total_time.append(time_end - time_start)

            c_final = QAOA_from_Ising(
                final_params, nlayers, portfolio_pauli_terms, portfolio_weights
            )
            probs = K.numpy(c_final.probability()).round(decimals=6)
            QAOA_loc = np.where(probs == np.max(probs))
            if QAOA_loc == loc:
                accuracy += 1 
        except:
            n_error += 1
            pass

        clear_output(wait=True)
        print("num of stocks:", num_assets)
        print("process: %d out of %d" % (loops, n_loops - 1))
        print("num of error:", n_error)
        print(result)
        print(total_time)

    time_avg = round(sum(total_time) / (n_loops - n_error), 4)
    result[num_assets] = {
        "time_avg": round(time_avg, 4),
        "accuracy": round(accuracy / (n_loops - n_error), 2),
        "variance": round(np.var(total_time) * 100, 4),
        "error": n_error
    }

num of stocks: 9
process: 9 out of 9
num of error: 2
{8: {'time_avg': 87.7903, 'accuracy': 0.12, 'variance': 666.091, 'num of error': 2}}
[124.03471398353577, 124.75357985496521, 128.2330949306488, 129.41070890426636, 126.1022891998291, 128.10096168518066, 129.5643630027771, 126.25299572944641]


## Results

In [4]:
result

{8: {'time_avg': 87.7903,
  'accuracy': 0.12,
  'variance': 666.091,
  'num of error': 2},
 9: {'time_avg': 127.0566,
  'accuracy': 0.25,
  'variance': 378.7215,
  'num of error': 2}}

In [ ]:
tc.about()